In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import randint

# Load datasets
holidays_events = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
sample_submission = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv')
stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')

# Merge train with holidays_events on date
merged_df = pd.merge(train, holidays_events, on='date', how='left')

# Merge the resulting dataframe with oil on date
merged_df = pd.merge(merged_df, oil, on='date', how='left')

# Merge the resulting dataframe with stores on store_nbr
merged_df = pd.merge(merged_df, stores, on='store_nbr', how='left')

# Merge the resulting dataframe with transactions on date and store_nbr
merged_df = pd.merge(merged_df, transactions, on=['date', 'store_nbr'], how='left')

# Display the first few rows of the merged dataframe to inspect the result
merged_df.tail()



,id,date,store_nbr,family,sales,onpromotion,type_x,locale,locale_name,description,transferred,dcoilwtico,city,state,type_y,cluster,transactions
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0
3054347,3000887,2017-08-15,9,SEAFOOD,16.000,0,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0


In [3]:
# Check for missing data in the merged dataframe
missing_data = merged_df.isnull().sum()

missing_data_percentage = (missing_data / len(merged_df)) * 100

missing_data_summary = pd.DataFrame({
    'Missing Values': missing_data,
    'Percentage': missing_data_percentage
})

missing_data_summary


,Missing Values,Percentage
id,0,0.000000
date,0,0.000000
store_nbr,0,0.000000
family,0,0.000000
sales,0,0.000000
onpromotion,0,0.000000
type_x,2551824,83.547258
locale,2551824,83.547258
locale_name,2551824,83.547258
description,2551824,83.547258


In [4]:
# Drop columns with excessive missing values
merged_df = merged_df.drop(columns=['type_x', 'locale', 'locale_name', 'description', 'transferred'])

# Fill missing values for dcoilwtico using forward fill
merged_df['dcoilwtico'] = merged_df['dcoilwtico'].fillna(method='ffill').fillna(method='bfill')

# Fill missing values for transactions using the median of each store
merged_df['transactions'] = merged_df.groupby('store_nbr')['transactions'].transform(lambda x: x.fillna(x.median()))

# Check the result
missing_data_summary_after = merged_df.isnull().sum().reset_index()
missing_data_summary_after.columns = ['Column', 'Missing Values']
missing_data_summary_after['Percentage'] = (missing_data_summary_after['Missing Values'] / len(merged_df)) * 100

missing_data_summary_after


/tmp/ipykernel_25/1175921753.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df['dcoilwtico'] = merged_df['dcoilwtico'].fillna(method='ffill').fillna(method='bfill')


,Column,Missing Values,Percentage
0,id,0,0.0
1,date,0,0.0
2,store_nbr,0,0.0
3,family,0,0.0
4,sales,0,0.0
5,onpromotion,0,0.0
6,dcoilwtico,0,0.0
7,city,0,0.0
8,state,0,0.0
9,type_y,0,0.0


In [5]:
# Convert date columns to datetime objects
merged_df['date'] = pd.to_datetime(merged_df['date'])

# Extract relevant features from dates
merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month
merged_df['day'] = merged_df['date'].dt.day
merged_df['day_of_week'] = merged_df['date'].dt.dayofweek

# Check the first few rows to ensure the transformations are correct
merged_df.tail()


,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,city,state,type_y,cluster,transactions,year,month,day,day_of_week
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,47.57,Quito,Pichincha,B,6,2155.0,2017,8,15,1
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,47.57,Quito,Pichincha,B,6,2155.0,2017,8,15,1
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,47.57,Quito,Pichincha,B,6,2155.0,2017,8,15,1
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,47.57,Quito,Pichincha,B,6,2155.0,2017,8,15,1
3054347,3000887,2017-08-15,9,SEAFOOD,16.000,0,47.57,Quito,Pichincha,B,6,2155.0,2017,8,15,1


In [6]:
#Convert date columns to datetime
merged_df['date'] = pd.to_datetime(merged_df['date'])
test['date'] = pd.to_datetime(test['date'])

In [7]:
merged_df.columns

Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion',
       'dcoilwtico', 'city', 'state', 'type_y', 'cluster', 'transactions',
       'year', 'month', 'day', 'day_of_week'],
      dtype='object')

In [8]:

# Create lag features for past sales
for lag in range(1, 8):
    merged_df[f'sales_lag_{lag}'] = merged_df.groupby(['store_nbr', 'family'])['sales'].shift(lag)

# Drop rows with NaN values generated by lag features
merged_df = merged_df.dropna()

# Extract date features
merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month
merged_df['day'] = merged_df['date'].dt.day
merged_df['day_of_week'] = merged_df['date'].dt.dayofweek

# Same for the test set
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['day_of_week'] = test['date'].dt.dayofweek

# Identify categorical columns
categorical_cols = ['family', 'city', 'state', 'type_y']

# Apply one-hot encoding
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_train = pd.DataFrame(encoder.fit_transform(merged_df[categorical_cols]))
encoded_train.columns = encoder.get_feature_names_out(categorical_cols)


# Drop original categorical columns and concatenate encoded columns
merged_df = merged_df.drop(columns=categorical_cols).reset_index(drop=True)
merged_df = pd.concat([merged_df, encoded_train], axis=1)

# Split the data into training and validation sets
X = merged_df.drop(columns=['id', 'date', 'sales'])
y = merged_df['sales']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest model using Grid Search
# Define a more simplified parameter grid
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10],
    'min_samples_split': [2],
    'min_samples_leaf': [1]
}

# Initialize the Random Forest model
rf = RandomForestRegressor(random_state=42)

# Perform Grid Search with the simplified grid
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Best model from Grid Search
best_rf = grid_search.best_estimator_

# Validate the model
y_val_pred = best_rf.predict(X_val)
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f'Validation RMSE: {val_rmse}')


Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=13.2min
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=24.9min
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=25.3min
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=13.1min
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=26.2min
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=45.3min
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=13.1min
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=26.3min
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=46.0min
[CV] END max_depth=5, min_samples_leaf=1,

In [9]:
val_rmse

247.63854905260627